REQUIREMENT

In [2]:
!pip install transformers torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [3]:
!pip install summa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54386 sha256=768f47f7be482520612d1af827aaa6d6558be1d1b77d29deba8a87e7c54a5382
  Stored in directory: /root/.cache/pip/wheels/4a/ca/c5/4958614cfba88ed6ceb7cb5a849f9f89f9ac49971616bc919f
Successfully built summa


In [4]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.2 MB/s eta 0:00:00


In [5]:
pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00


INITIALIZE


In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from bs4 import BeautifulSoup
import requests
import torch
from summa import summarizer
import evaluate

In [7]:
device = "cpu" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen1.5-0.5B-Chat"
)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-0.5B-Chat")

#model = AutoModelForQuestionAnswering.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


SIMPLE EXAMPLE

In [8]:
def generate_response(model, tokenizer, device, prompt, web_page_text="", max_new_tokens=512):
    # Define the messages
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": web_page_text},
        {"role": "user", "content": prompt}
    ]

    # Apply chat template and tokenize the messages
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    # Generate response using the model
    generated_ids = model.generate(model_inputs.input_ids, max_new_tokens=max_new_tokens)

    # Decode the generated ids to obtain the response
    response = tokenizer.decode(generated_ids[0][len(model_inputs.input_ids[0]):], skip_special_tokens=True)

    return response

In [9]:
prompt = "How is the water composed of?"
response = generate_response(model, tokenizer, device, prompt)
print(response)

Water is composed primarily of hydrogen atoms and oxygen atoms, along with other small amounts of other elements such as nitrogen, phosphorus, and chlorine. These atoms combine to form water molecules that can exist in different forms depending on their chemical properties. For example, water can be dissolved in saltwater or ice, or it can exist as a solid and float in water. The exact composition of water is not well understood, but it is widely considered one of the most abundant and versatile substances on Earth.


WEB CONTEXT

In [10]:
responses = []

In [11]:
def extract_text_from_web_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Rimuovi gli elementi non desiderati come script, stili, commenti, ecc.
    for script in soup(["script", "style", "nav", "footer", "header","h1", "h2", "h3", "h4", "h5", "h6", "title", "a", "button"]):
        script.extract()

    # Estrai il testo rimanente
    text = soup.get_text()

    # Elimina le linee vuote e le righe indesiderate
    lines = (line.strip() for line in text.splitlines())
    clean_text = '\n'.join(line for line in lines if line)

    return clean_text

In [12]:
# URL della pagina web da cui estrarre il testo
url = "https://www.unipd.it/en/how-apply"
#max_text_length = 1000
web_page_text = extract_text_from_web_page(url)
#print(web_page_text)
# Summarize the extracted text


QUESTION 1

In [13]:
prompt = "Can I apply for the admission of the university of Padua if I am a non-italian student?"
response = generate_response(model, tokenizer, device, prompt, web_page_text)
print(response)
responses.append(response)

Yes, you can apply for admission to the University of Padua if you are not an Italian student. To be eligible for admission, you will need to meet certain criteria, including having a minimum 12-year school record and being an international student holding a non-Italian academic diploma, degree or equivalent.


QUESTION 2

In [14]:
prompt = "Are the courses provided the university of Padua taught only in Italian?"
response = generate_response(model, tokenizer, device, prompt, web_page_text)
print(response)
responses.append(response)

No, the courses provided by the University of Padua cover both Italian and foreign languages, and are taught in English. The university provides various courses with the aim to train students in various fields, including business, engineering, social sciences, humanities, and arts. These courses are designed to equip students with the skills required to pursue their career goals and prepare them for success in their respective industries.


QUESTIOIN 3

In [15]:
prompt = "I missed the deadlines for submitting application for the first semester. Can I enroll and submit the application for the second (winter) semester?"
response = generate_response(model, tokenizer, device, prompt, web_page_text)
print(response)
responses.append(response)

Yes, you can enroll and submit the application for the second (winter) semester. However, please keep in mind that there may be additional deadlines for both semesters.

- The first semester has a maximum of 2 months to apply and complete your application.
- The second semester typically starts in December or January.

If you miss the deadline for either semester, you will need to wait until the last day of the next semester to apply again.

You can check the deadlines for both semesters by checking the respective program's website or contacting the program coordinator/secretariat. They should be able to guide you through the process of submitting your application and providing additional information about the deadlines for the respective semesters.


QUESTION 4

In [16]:
prompt = "I want to apply for the bachelor's degree in biology, how to know if I am eligible?"
response = generate_response(model, tokenizer, device, prompt, web_page_text)
print(response)
responses.append(response)

If you are interested in obtaining a bachelor's degree in biology at the University of Padua, there are several steps you can follow:

  1. Determine your eligibility criteria: Before applying for a program, make sure you meet the following eligibility criteria:

  * Have at least a high school diploma, degree or equivalent
  * Be eligible for a Bachelor's degree or a single-cycle degree
  * Hold a minimum 12-year school record

  2. Review the application process: If you are applying for a bachelor's degree, you will need to review the application process outlined in the program guidelines. This may include reviewing the application form, answering questions about your background, and meeting the program requirements.
  3. Apply for a single-cycle degree: If you are applying for a single-cycle degree, you will need to meet the application requirements listed in the program guidelines. These requirements typically include a minimum 12-year school record and passing a series of exams.
 

QUESTION 5

In [17]:
prompt = "The duration of diploma that I obtained is 11 years in my country, does that mean that I can not apply for the admission of the bachelor's degree program in biology of the university of Padua?"
response = generate_response(model, tokenizer, device, prompt, web_page_text)
print(response)
responses.append(response)

No, your 11-year-old school record does not mean that you cannot apply for the Bachelor's degree program in biology of the University of Padua. However, you will still need to meet certain eligibility criteria.
To be eligible for admission to a Bachelor's degree program in biology, you must have attended at least 1 (in case of 11 years of schooling) or 2 (in case of 10 years of schooling) years of academic studies and passed all the scheduled exams.
Your high school diploma may still be required for admission to most programs in biology, as it is a crucial element of academic success and knowledge transfer.
It is important to note that not all programs require applicants to attend at least 12 years of schooling, and there are many other factors that may affect the ability to apply for admission, such as grades, extracurricular activities, personal qualifications, and financial aid.
Therefore, while your high school diploma may be a factor that may help you stand out during the applicat

QUESTION 6

In [18]:
prompt = "How can I make sure if my previous university which is not Italian is qualified in order to apply for the master's degree programs?"
response = generate_response(model, tokenizer, device, prompt, web_page_text)
print(response)
responses.append(response)

You can follow these steps to ensure that your previous university is qualified:

  1. Check the national accreditation system of the country where your previous university is located. This will help you determine if your institution is meeting the requirements set by the program you want to apply for.
  2. Look for other relevant certifications or qualifications that your previous institution may have earned to meet the program requirements. For example, if your previous university was accredited by an organization that assesses the quality of international education, you may want to consider checking its academic reputation or the number of degrees it has received.
  3. Contact the program coordinator/secretariat to ask about their资格 for the specific program you want to apply for. They should be able to provide information about their credentials and their ability to provide any necessary certifications or qualifications.
  4. If you still need help evaluating the资格 of your previous 

QUESTION 7

In [19]:
prompt = "Can I apply for just single course units not the whole degree programs of the University of Padua?"
response = generate_response(model, tokenizer, device, prompt, web_page_text)
print(response)
responses.append(response)

Yes, you can apply for only single course units. The singlecourse units refer to courses that are designed specifically for individuals who wish to learn a particular subject without pursuing a full-time degree program at the university.
Singlecourse units can be taken in various fields, including computer science, business, engineering, social sciences, etc. Each course can cover only one or two of the specific areas that you are interested in learning, rather than taking multiple courses at the university.
To apply for singlecourse units, you will need to ensure that you have completed all the required courses for that particular area of study. You can then check the course schedule of the university and find out if there are any special opportunities available for singlecourse units.
It's important to note that the singlecourse units offered by the university may differ slightly from those offered by other institutions, so you should review the details of each course carefully befor

QUESTION 8

In [20]:
prompt = "My Bachelor's degree was a double degree program obtained from two different universities, one from Indonesia and one from Singapore, how does this affect my application for admission for the master's degree programs?"
response = generate_response(model, tokenizer, device, prompt, web_page_text)
print(response)
responses.append(response)

If you have obtained both degrees from different countries, it is possible that your application may be rejected for certain programs due to differences in cultural or academic standards between the two countries. In addition, if you do not have the relevant qualifications or experience required for admission to a Master's degree program from a particular country, you may also face difficulties in gaining acceptance. 

In general, the most important factors that apply to admission to a Master's degree program from a different country will depend on the specific program, the country in which you plan to study, and your personal goals and interests. Therefore, it is important to carefully research each program and consider all the factors before making a decision.

If you are considering applying for a Master's degree program from Indonesia, you may want to reach out to the program coordinator or secretariat at the University of Padua for more information about the admission process and 

QUESTION 9

In [21]:
prompt = "what are the eligibilities to apply for the admission to a master's degree program?"
response = generate_response(model, tokenizer, device, prompt, web_page_text)
print(response)
responses.append(response)

The application process for a Master's degree program varies depending on the program being offered and the country in which the program is located. However, generally, there are several eligibility criteria that applicants should meet to be considered for admission to a Master's degree program.

Here are some common eligibility criteria for Master's degree programs:

  1. Bachelor's degree in a field relevant to the field of study.
  2. English proficiency with a score above 3.5 in the TOEFL (Test of English Language proficiency) orIELTS (International English Language Testing System).
  3. Academic track record, including completed degrees, thesis projects, or research projects.
  4. Experience or training, such as work experience, internships, or tutoring.
  5. Proof of financial support, such as scholarships, grants, or loans.
  6. Proficiency in a foreign language.
  7. Good communication skills, particularly in written and verbal communication.
  8. Personal qualities and interes

QUESTION 10

In [22]:
prompt = "Can you give the contact info so I can get more detailed academic information?"
response = generate_response(model, tokenizer, device, prompt, web_page_text)
print(response)
responses.append(response)

Certainly! The Contact Information section of the University of Padua website outlines the various ways to contact the university and get further information about its academic programs and services.
For inquiries regarding degree programs:

  * Enrolling in a Master's program: Check the website for information on the entry requirements and deadline for applications, including specific courses.
  * Enrolling in a Bachelor's program: Check the website for information on the entry requirements and deadline for applications, including specific courses.
  * Enrolling in an English-taught degree course: Check the website for information on the entry requirements and deadline for applications.

For inquiries regarding academic-related information:

  * Academic related services: Call the program coordinator/secretariat or via email if you need assistance with enrolling in a degree program, academic related services, or other related issues.
  * Academic related resources: Check the program's

In [23]:
expected_answers = ["yes, you can. The University of Padua welcomes applications from international students. You are considered an “international student” if you hold a non-Italian academic diploma, degree or equivalent.",
                    "No. There are two type of courses; ENGLISH-taught degree courses and ITALIAN-taught degree courses ",
                    "No, unfortunately you can not. courses at the University of Padua start at the end of September/beginning of October. Enrolment for the 2nd semester is not available.",
                    "To be eligible for admission to a Bachelor’s degree or a single-cycle degree, you need a minimum 12-year school record. If you have been awarded a final diploma after a shorter programme of study (i.e. less than 12 years), in order to become eligible you must: have attended at least 1 (in case of 11 years of schooling) or 2 (in case of 10 years of schooling) years of academic studies and passed all the scheduled exams. OR have obtained a post-secondary qualification  in a non-University higher education institution, certifying the additional one or two years of education. OR have successfully completed one or two year preparatory courses (Foundation courses) to compensate for the missing years of schooling.",
                    "you must take additional courses to become eligible for admission to a Bachelor's degree program. If you have been awarded a final diploma after a shorter programme of study (i.e. less than 12 years), in order to become eligible you must: have attended at least 1 (in case of 11 years of schooling) or 2 (in case of 10 years of schooling) years of academic studies and passed all the scheduled exams. OR have obtained a post-secondary qualification  in a non-University higher education institution, certifying the additional one or two years of education. OR have successfully completed one or two year preparatory courses (Foundation courses) to compensate for the missing years of schooling. Or as an alternative, you must: have attended an additional one (in case of 12 years of schooling) or two (in case of 11 years of schooling) years of academic studies in another country and/or different foreign system and passed all the scheduled exams. OR have obtained an official Italian or foreign post-secondary qualification in a non-University higher education institution with a minimum duration of one or two years. OR have successfully completed Foundation courses held from an Italian University. These additional certificates only allow enrolment to the first year and cannot be further evaluated for shortening of your degree programme",
                    "You can check a country-based list of qualifications in order to verify whether their entry qualification is suitable to access your degree programme of interest.",
                    "yes, you can. If you are you interested in attending single course units and take the corresponding exams without having to be enrolled in any degree course, visit the dedicated page.",
                    "If your bachelor's degree was issued by two or more universities (e.g. the awarding and the teaching institutions are not the same and pertain to different national education systems), each institution must be accredited according to the national educational accreditation system of the country where it is located..",
                    "To be eligible for admission to a Master’s degree, you need a Bachelor’s degree or equivalent foreign qualification with specific curricular requirements obtained from a higher education institution accredited according to the national educational accreditation system of the country where it is located.",
                    "sure,For academic related information you can contact the programme coordinator/secretariat tel. +39 0498273131, or send an email to urp@unipd.it "]

RAGAS


In [24]:
pip install -U deepeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.6/245.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 

In [25]:
import os
from getpass import getpass

In [26]:
# Get the API token from the user
OPENAI_API_KEY = getpass("Enter your openai API token: ")

# Set the environment variable
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

Enter your openai API token: ··········


In [27]:
from deepeval import evaluate
from deepeval.metrics.ragas import RagasMetric
from deepeval.test_case import LLMTestCase

# Replace this with the actual output from your LLM application
actual_outputs = responses
print(len(responses))
# Replace this with the expected output from your RAG generator
expected_outputs = expected_answers
print(len(expected_answers))

10
10


In [28]:
# Replace this with the actual retrieved context from your RAG pipeline
retrieval_contexts = [
    "The University of Padua welcomes applications from international students. You are considered an “international student” if you hold a non-Italian academic diploma, degree or equivalent.",
    "There are two types of degree courses; ENGLISH-taught degree courses: check application procedures and the course factsheets for detailed information. In order to submit your application (except for the Single-cycle degree programme in Medicine and Surgery), please go to: apply.unipd.it ITALIAN-taught degree courses: applications dates vary depending on the course - check application procedures",
    "Courses at the University of Padua start at the end of September/beginning of October. Enrolment for the 2nd semester is not available.",
    "To be eligible for admission to a Bachelor’s degree or a single-cycle degree, you need a minimum 12-year school record. If you have been awarded a final diploma after a shorter programme of study (i.e. less than 12 years), in order to become eligible you must: have attended at least 1 (in case of 11 years of schooling) or 2 (in case of 10 years of schooling) years of academic studies and passed all the scheduled exams. OR have obtained a post-secondary qualification  in a non-University higher education institution, certifying the additional one or two years of education. OR have successfully completed one or two year preparatory courses (Foundation courses) to compensate for the missing years of schooling.",
    "To be eligible for admission to a Bachelor’s degree or a single-cycle degree, you need a minimum 12-year school record. If you have been awarded a final diploma after a shorter programme of study (i.e. less than 12 years), in order to become eligible you must: have attended at least 1 (in case of 11 years of schooling) or 2 (in case of 10 years of schooling) years of academic studies and passed all the scheduled exams. OR have obtained a post-secondary qualification  in a non-University higher education institution, certifying the additional one or two years of education. OR have successfully completed one or two year preparatory courses (Foundation courses) to compensate for the missing years of schooling. As an alternative to this certification, you must: have attended an additional one (in case of 12 years of schooling) or two (in case of 11 years of schooling) years of academic studies in another country and/or different foreign system and passed all the scheduled exams. OR have obtained an official Italian or foreign post-secondary qualification in a non-University higher education institution with a minimum duration of one or two years. OR have successfully completed Foundation courses held from an Italian University. These additional certificates only allow enrolment to the first year and cannot be further evaluated for shortening of your degree programme",
    "Foreign entry qualifications suitable for admission; Applicants can consult a country-based list of qualifications in order to verify whether their entry qualification is suitable to access their degree programme of interest..",
    "Apply for a single course unit ; If you are you interested in attending single course units and take the corresponding exams without having to be enrolled in any degree course, visit the dedicated page.",
    "Should your study title be issued by two or more universities (e.g. the awarding and the teaching institutions are not the same and pertain to different national education systems), each institution must be accredited according to the national educational accreditation system of the country where it is located.",
    "To be eligible for admission to a Master’s degree, you need a Bachelor’s degree or equivalent foreign qualification with specific curricular requirements obtained from a higher education institution accredited according to the national educational accreditation system of the country where it is located.",
    "Global Engagement Office - Recruitment Unit Palazzo Anselmi Lungargine del Piovego 1 - 35131 Padova Get in touch with the Global Engagement Office For academic related information, please contact the programme coordinator/secretariat tel. +39 0498273131 or send an email to  urp@unipd.it "
]


test_cases = []

metric = RagasMetric(threshold=0.6, model="gpt-4o")

# Loop through each question and create a test case
for idx, (actual_output, expected_output, retrieval_context) in enumerate(zip(actual_outputs, expected_outputs, retrieval_contexts)):
    test_case = LLMTestCase(
        input=f"Question {idx + 1}",
        actual_output=actual_output,
        expected_output=expected_output,
        retrieval_context=[retrieval_context]  # Wrap the string in a list
    )
    test_cases.append(test_case)

# Evaluate test cases
metrics = evaluate(test_cases, [metric])

Output()

Evaluating test cases...
Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]



Metrics Summary

  - ✅ RAGAS (score: 0.7543007763667897, threshold: 0.6, strict: False, evaluation model: gpt-4o, reason: None, error: None)
      - Contextual Precision (ragas) (score: 0.9999999999)
      - Contextual Recall (ragas) (score: 0.6666666666666666)
      - Faithfulness (ragas) (score: 0.6)
      - Answer Relevancy (ragas) (score: 0.7505364389004923)

For test case:

  - input: Question 1
  - actual output: Yes, you can apply for admission to the University of Padua if you are not an Italian student. To be eligible for admission, you will need to meet certain criteria, including having a minimum 12-year school record and being an international student holding a non-Italian academic diploma, degree or equivalent.
  - expected output: yes, you can. The University of Padua welcomes applications from international students. You are considered an “international student” if you hold a non-Italian academic diploma, degree or equivalent.
  - context: None
  - retrieval context: [

/usr/local/lib/python3.10/dist-packages/portalocker/utils.py:218: UserWarning: timeout has no effect in blocking mode
  warnings.warn(


✅ Tests finished! Run "deepeval login" to view evaluation results on the web.

In [31]:
from statistics import mean

# puting the results of metrics in test_results to avoid confusion
test_results = metrics
# Initialize lists to collect scores for each metric and overall scores
contextual_precision_scores = []
contextual_recall_scores = []
faithfulness_scores = []
answer_relevancy_scores = []
overall_scores = []

# Iterate through each test result to extract the scores from score_breakdown and overall score
for test_result in test_results:
    # Access the RagasMetric object
    ragas_metric = test_result.metrics[0]

    # Access the score_breakdown dictionary
    score_breakdown = ragas_metric.score_breakdown

    # Append the scores to the respective lists
    contextual_precision_scores.append(score_breakdown['Contextual Precision (ragas)'])
    contextual_recall_scores.append(score_breakdown['Contextual Recall (ragas)'])
    faithfulness_scores.append(score_breakdown['Faithfulness (ragas)'])
    answer_relevancy_scores.append(score_breakdown['Answer Relevancy (ragas)'])

    # Append the overall score to the overall_scores list
    overall_scores.append(ragas_metric.score)

# Calculate the average for each metric
avg_contextual_precision = mean(contextual_precision_scores)
avg_contextual_recall = mean(contextual_recall_scores)
avg_faithfulness = mean(faithfulness_scores)
avg_answer_relevancy = mean(answer_relevancy_scores)
avg_overall_score = mean(overall_scores)

# Print the average scores

print("Qwen/Qwen1.5-0.5B-Chat:")
print(f"Contextual Precision (average score: {avg_contextual_precision})")
print(f"Contextual Recall (average score: {avg_contextual_recall})")
print(f"Faithfulness (average score: {avg_faithfulness})")
print(f"Answer Relevancy (average score: {avg_answer_relevancy})")
print(f"RAGAS (average overall score: {avg_overall_score})")

Qwen/Qwen1.5-0.5B-Chat:
Contextual Precision (average score: 0.9999999999)
Contextual Recall (average score: 0.8333333333333334)
Faithfulness (average score: 0.1902941176470588)
Answer Relevancy (average score: 0.6758084369400539)
RAGAS (average overall score: 0.6748589719551115)
